In [1]:
import pandas as pd
import numpy as np
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/25_KingsOfML')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# df_train=pd.read_csv('01.RawData/train.csv')
# df_test=pd.read_csv('01.RawData/test_BDIfz5B.csv')
# df_cmpgn=pd.read_csv('01.RawData/campaign_data.csv')
# df_sample=pd.read_csv('01.RawData/sample_submission_4fcZwvQ.csv')
# df_train['send_date']=pd.to_datetime(df_train['send_date'],format='%d-%m-%Y %H:%M')
# df_test['send_date']=pd.to_datetime(df_test['send_date'],format='%d-%m-%Y %H:%M')
# df_train.sort_values(by=['user_id','send_date'],inplace=True)
# df_test.sort_values(by=['user_id','send_date'],inplace=True)
# store=pd.HDFStore('01.RawData/DataStore.h5')
# store['train']=df_train
# store['test']=df_test
# store['cmpgn']=df_cmpgn
# store['sample']=df_sample
# store.close()

df_train=pd.read_hdf('01.RawData/DataStore.h5', 'train')
df_test=pd.read_hdf('01.RawData/DataStore.h5', 'test')
df_cmpgn=pd.read_hdf('01.RawData/DataStore.h5', 'cmpgn')
df_sample=pd.read_hdf('01.RawData/DataStore.h5', 'sample')

In [7]:
def get_train_test_campaigns(valid=True):
    train_months=[8,9] if valid==True else [8,9,10,11,12]
    valid_months=[10,11,12] if valid==True else [13]
    train_campaigns=master[master.month.isin(train_months)].index.get_level_values(1).drop_duplicates()
    valid_campaigns=master[master.month.isin(valid_months)].index.get_level_values(1).drop_duplicates()
    return train_campaigns.tolist(),valid_campaigns.tolist()
  
df_train['source']='train'
df_train['month']=df_train['send_date'].dt.month
df_test['source']='test'
df_test['month']=13
master=pd.concat([df_train,df_test])
master.set_index(['user_id','campaign_id','id'],inplace=True)
train_campaigns,valid_campaigns=get_train_test_campaigns(valid=True)
master=master[master.index.get_level_values(1).isin(train_campaigns+valid_campaigns)]
master['ds']=np.where(master.index.get_level_values(1).isin(train_campaigns),'1.Train','2.Valid')
master.sort_values(by='ds',inplace=True)
master['rcv']=1

In [79]:
user_tags=master[master['ds']=='1.Train'].reset_index().groupby(['user_id'])[['rcv','is_open','is_click']].max()
user_tags.rename(columns={'rcv':'FlagRcvEarlier','is_open':'OpenActiveFlag','is_click':'ClickActiveFlag'},inplace=True)
valid_cmpgn_vars=pd.merge(master[master['ds']=='2.Valid'].reset_index(),user_tags.reset_index(),on='user_id',how='left')
valid_cmpgn_vars=valid_cmpgn_vars.fillna(0)

In [80]:
vars1=['rcv','FlagRcvEarlier','OpenActiveFlag','ClickActiveFlag','is_click','is_open']
valid_cmpgn_vars=valid_cmpgn_vars.groupby('campaign_id')[vars1].sum()
valid_cmpgn_vars['RcvEarlier']=1.0*valid_cmpgn_vars['FlagRcvEarlier']/valid_cmpgn_vars['rcv']
valid_cmpgn_vars['OpenActive']=1.0*valid_cmpgn_vars['OpenActiveFlag']/valid_cmpgn_vars['rcv']
valid_cmpgn_vars['ClickActive']=1.0*valid_cmpgn_vars['ClickActiveFlag']/valid_cmpgn_vars['rcv']
valid_cmpgn_vars['OpenRate']=1.0*valid_cmpgn_vars['is_open']/valid_cmpgn_vars['rcv']
valid_cmpgn_vars['ClickRate']=1.0*valid_cmpgn_vars['is_click']/valid_cmpgn_vars['rcv']

In [81]:
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import mean_squared_error

reg=Lasso(positive=True,alpha=0)
reg.fit(valid_cmpgn_vars[['RcvEarlier','OpenActive','ClickActive']],valid_cmpgn_vars['ClickRate'])
valid_cmpgn_vars['prediction']=reg.predict(valid_cmpgn_vars[['RcvEarlier','OpenActive','ClickActive']])
print reg.intercept_, reg.coef_
print mean_squared_error(valid_cmpgn_vars['ClickRate'],valid_cmpgn_vars['prediction'])

#this shows that campaign click rate depends only on #Customers who are click active
#next, we make find if openn rates depends on Single Open/Multiple Opens

0.00120036959423 [ 0.         0.         0.3703789]
3.69872087765e-06


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  """


In [82]:
user_tags=master[master['ds']=='1.Train'].reset_index().groupby(['user_id'])['is_click'].sum()
valid_cmpgn_vars=master[master['ds']=='2.Valid'].reset_index()
valid_cmpgn_vars['NumEmailsClicked']=valid_cmpgn_vars['user_id'].map(user_tags).fillna(0)
valid_cmpgn_vars['EmailClicked']=np.clip(valid_cmpgn_vars['NumEmailsClicked'],0,2)
valid_cmpgn_vars['MultEmailClicked']=(valid_cmpgn_vars['NumEmailsClicked']>1).astype(np.int64)
valid_cmpgn_vars=valid_cmpgn_vars.groupby('campaign_id')[['rcv','EmailClicked','MultEmailClicked','is_click']].sum()
valid_cmpgn_vars['EmailClickedRate']=1.0*valid_cmpgn_vars['EmailClicked']/valid_cmpgn_vars['rcv']
valid_cmpgn_vars['MultEmailClickedRate']=1.0*valid_cmpgn_vars['MultEmailClicked']/valid_cmpgn_vars['rcv']
valid_cmpgn_vars['ClickRate']=1.0*valid_cmpgn_vars['is_click']/valid_cmpgn_vars['rcv']


In [85]:
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import mean_squared_error

reg=Lasso(positive=True,alpha=0)
reg.fit(valid_cmpgn_vars[['EmailClickedRate','MultEmailClickedRate']],valid_cmpgn_vars['ClickRate'])
valid_cmpgn_vars['prediction']=reg.predict(valid_cmpgn_vars[['EmailClickedRate','MultEmailClickedRate']])
print reg.intercept_, reg.coef_
print mean_squared_error(valid_cmpgn_vars['ClickRate'],valid_cmpgn_vars['prediction'])*1000
#The estimates here will be error prone. 
#To make them more reliable, calculate variables for each campaign separately
valid_cmpgn_vars

0.00586654274138 [ 0.          2.20339876]
0.00249438232204


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  """


,rcv,EmailClicked,MultEmailClicked,is_click,EmailClickedRate,MultEmailClickedRate,ClickRate,prediction
campaign_id,,,,,,,,
48,51456,1412.0,109,626.0,0.027441,0.002118,0.012166,0.010534
50,39710,2058.0,204,574.0,0.051826,0.005137,0.014455,0.017186
51,3882,503.0,66,170.0,0.129572,0.017002,0.043792,0.043328
52,82160,2778.0,238,1166.0,0.033812,0.002897,0.014192,0.012249
53,85431,2166.0,188,862.0,0.025354,0.002201,0.010090,0.010715
54,93943,2588.0,216,963.0,0.027549,0.002299,0.010251,0.010933


In [44]:
valid_cmpgn_vars

,rcv,EmailClicked,MultEmailClicked,is_click,EmailClickedRate,MultEmailClickedRate,ClickRate,prediction
campaign_id,,,,,,,,
48,51456,1412.0,109,626.0,0.027441,0.002118,0.012166,0.010534
50,39710,2058.0,204,574.0,0.051826,0.005137,0.014455,0.017186
51,3882,503.0,66,170.0,0.129572,0.017002,0.043792,0.043328
52,82160,2778.0,238,1166.0,0.033812,0.002897,0.014192,0.012249
53,85431,2166.0,188,862.0,0.025354,0.002201,0.010090,0.010715
54,93943,2588.0,216,963.0,0.027549,0.002299,0.010251,0.010933


In [90]:
t1=df_train.reset_index().set_index(['user_id','campaign_id'])
t1['rcv']=1
t1=t1[['rcv','is_open','is_click']]
t1=t1.stack().unstack(1).fillna(0).cumsum(axis=1)
t1=t1.stack().unstack(1).reset_index()
t1['campaign_id']=t1['campaign_id']+1
master1=pd.merge(df_train[df_train['campaign_id']>=45].reset_index(),t1,on=['user_id','campaign_id'])


In [119]:
master1['flag_rcv_1']=(master1['rcv']>=1).astype(np.int64)
master1['flag_rcv_2']=(master1['rcv']>=2).astype(np.int64)
master1['flag_open_1']=(master1['is_open_y']>=1).astype(np.int64)
master1['flag_open_2']=(master1['is_open_y']>=2).astype(np.int64)
master1['flag_click_1']=(master1['is_click_y']==1).astype(np.int64)
master1['flag_click_2']=(master1['is_click_y']>=2).astype(np.int64)
vars1=['flag_rcv_1','flag_click_1','flag_click_2','is_click_x']
campaign_vars=master1.groupby(['campaign_id'])[['rcv']+vars1].sum()
for var in vars1:
    campaign_vars[var]=1.0*campaign_vars[var]/campaign_vars['rcv']
    


In [122]:
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import mean_squared_error
# reg=Lasso(positive=True,alpha=0)
reg=LinearRegression()
reg.fit(campaign_vars[vars1[:-1]],campaign_vars[vars1[-1]])
campaign_vars['prediction']=reg.predict(campaign_vars[vars1[:-1]])
print reg.intercept_, reg.coef_
print mean_squared_error(campaign_vars[vars1[-1]],campaign_vars['prediction'])
#The estimates here will be error prone. 
#To make them more reliable, calculate variables for each campaign separately
campaign_vars

-0.0164045676383 [ 0.0818122  -0.19894723  3.14457085]
4.68210756935e-06


,rcv,flag_rcv_1,flag_click_1,flag_click_2,is_click_x,prediction
campaign_id,,,,,,
45,28173.0,0.188514,0.015831,0.003940,0.013488,0.008258
46,38070.0,0.246520,0.011164,0.001629,0.007118,0.006664
47,68742.0,0.199383,0.015391,0.002808,0.003462,0.005674
48,206268.0,0.231117,0.008911,0.001299,0.003035,0.004817
49,372615.0,0.214806,0.009098,0.001449,0.002786,0.003916
50,214597.0,0.184243,0.010811,0.002265,0.002675,0.003639
51,29392.0,0.128164,0.016943,0.005580,0.005784,0.008256
52,443268.0,0.182673,0.008943,0.001631,0.002630,0.001890
53,407754.0,0.167243,0.008677,0.001795,0.002114,0.001197


In [124]:
t1=df_train.copy()
t1['month']=t1['send_date'].dt.month
t1['rcv']=1
t2=t1[t1['month']<10].groupby('user_id')[['rcv','is_open','is_click']].max().add_prefix('ever_')
t3=t1[t1['month']==9].groupby('user_id')[['rcv','is_open','is_click']].max().add_prefix('prev_')

t4=pd.merge(t1[t1['month']>=10],t2,left_on='user_id',right_index=True,how='left').fillna(0)
t5=pd.merge(t4,t3,left_on='user_id',right_index=True,how='left').fillna(0)
t5.groupby(['ever_rcv','ever_is_open','ever_is_click','prev_rcv','prev_is_open','prev_is_click'])[['rcv','is_open','is_click']].sum().to_clipboard()